In [11]:
import pandas as pd
import numpy as np
gen = pd.read_csv("./TX_Data/SourceData/gen.csv")
#uc = pd.read_csv("/Users/ethan/Dev/PERFORM/Prescient/downloads/rts_gmlc/RTS-GMLC/RTS_Data/FormattedData/PLEXOS/PLEXOS_Solution/DAY_AHEAD Solution Files/noTX/PLEXOS_DA_solution_commitment.csv", index_col='time')
#uc.shape

def calc_on_time(uc):
    diff = uc.diff()
 
    initial_state_dict = dict()
    for col in diff.columns:
        s = diff[col].dropna()
        diff_s = s[s!=0]
        if diff_s.empty:
            check_row = uc[col].head(1)
        else:
            check_row = diff_s.tail(1)
 
        if check_row.values == -1 or check_row.values == 0:
            initial_state_dict[col] = -1 * (len(uc.index) - uc.index.get_loc(check_row.index[0]))
        else:
            initial_state_dict[col] = len(uc.index) - uc.index.get_loc(check_row.index[0])
        #print(len(uc.index))
        #print(uc.index.get_loc(check_row.index[0]))
    
    return(initial_state_dict)

In [25]:
lst = []
cols = gen['GEN UID'].tolist()
# cols.insert(0, 'datetime')
# go through the date range and assign half the assets to be on randomly, solar assets all on between 8am and 8pm
for i in pd.date_range(start="2018-07-05", end="2018-07-10", freq="H"):   
    new_list = []
    counter = 0
    # select which generators are on
    if(i.hour == 0):
        gen_on = np.random.choice(2, size = len(gen[gen["Fuel"]!="SUN (Solar)"].index))
    for asset in gen["Fuel"]:
        if(asset == "SUN (Solar)"):
            if(i.hour >= 8 and i.hour <= 16):
                new_list.append(1)
            else:
                new_list.append(0)
        else:
            if(gen_on[counter]):
                new_list.append(1)
            else:
                new_list.append(0)
            counter += 1
    lst.append(new_list)
       
uc = pd.DataFrame(lst, columns = cols)
#uc.set_index("datetime")
on_time = calc_on_time(uc)

In [26]:
import csv

with open('./TX_Data/FormattedData/PLEXOS/PLEXOS_Solution/DAY_AHEAD Solution Files/noTX/on_time_7.10.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, on_time.keys())
    w.writeheader()
    w.writerow(on_time)